In [6]:
root='/kaggle/working/'
import sys
module_path = root+'thesis/src/EDA'
sys.path.append(module_path)
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import random
from scipy.stats import chi2
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
import pandas as pd
!pip install fastparquet
!pip install gdown

Questa è un analisi per capire su quanti utenti abbiamo un vero e proprio longitudinale di commenti. 
Si definisce longitudinale un dato se abbiamo almento 50 commenti, con un rate di 2 commenti per mese in media.

## Facebook

In [9]:
# Scarica i tre dataset che appartengono a Facebook
import gdown
url='https://drive.google.com/uc?id=1Y2lGWkcgo_IWHdWFh_Qcn0K74D_xQhvB'
output='facebook_news.csv'
gdown.download(url,output,quiet=False)

url='https://drive.google.com/uc?id=1KGH2s8sPnvXZc7_wVgTCoGTXPr3B7SCe'
output='facebook_brexit.csv'
gdown.download(url,output,quiet=False)

url='https://drive.google.com/uc?id=17q4X4JHYJBz1jF58XvYfbKqZLy-jtmzM'
output='facebook_vaccines.csv'
gdown.download(url,output,quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1Y2lGWkcgo_IWHdWFh_Qcn0K74D_xQhvB
From (redirected): https://drive.google.com/uc?id=1Y2lGWkcgo_IWHdWFh_Qcn0K74D_xQhvB&confirm=t&uuid=228dad60-15e0-49e1-a331-98711fa799d3
To: /kaggle/working/facebook_news.csv
100%|██████████| 1.84G/1.84G [00:18<00:00, 98.1MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1KGH2s8sPnvXZc7_wVgTCoGTXPr3B7SCe
From (redirected): https://drive.google.com/uc?id=1KGH2s8sPnvXZc7_wVgTCoGTXPr3B7SCe&confirm=t&uuid=708e1bb2-748b-427c-a4ef-35612566d704
To: /kaggle/working/facebook_brexit.csv
100%|██████████| 119M/119M [00:02<00:00, 50.0MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=17q4X4JHYJBz1jF58XvYfbKqZLy-jtmzM
From (redirected): https://drive.google.com/uc?id=17q4X4JHYJBz1jF58XvYfbKqZLy-jtmzM&confirm=t&uuid=9308b4da-2ab6-47d4-85b0-856918ec259a
To: /kaggle/working/facebook_vaccines.csv
100%|██████████| 599M/599M [00:15<00:00, 39.5MB/s] 


'facebook_vaccines.csv'

In [30]:
# Leggi i tre dataset con pandas
facebook_news = pd.read_csv('/kaggle/working/facebook_news.csv', dtype={'from_id': str})
facebook_vaccines_all_comments_scored = pd.read_csv('/kaggle/working/facebook_vaccines.csv', dtype={'from_id': str})
facebook_brexit_all_comments_scored = pd.read_csv('/kaggle/working/facebook_brexit.csv', dtype={'from_id': str})

# Etichetta e raggruppa i tre dataset
facebook_vaccines = facebook_vaccines_all_comments_scored.copy()
facebook_vaccines['topic'] = 'Vaccines'

facebook_brexit= facebook_brexit_all_comments_scored.copy()
facebook_brexit['topic'] = 'Brexit'

facebook_news = facebook_news.copy()
facebook_news['topic'] = 'News'

facebook = pd.concat([facebook_vaccines, facebook_brexit, facebook_news])
facebook.reset_index(drop=True, inplace=True)

In [69]:
import pandas as pd

# Assumendo che 'facebook' è il tuo DataFrame e 'created_time' è già in formato datetime
facebook['created_time'] = pd.to_datetime(facebook['created_time'])

# Conta i commenti per ciascun utente
user_counts = facebook['from_id'].value_counts()

# Filtra gli utenti con almeno 50 commenti
users_at_least_50 = user_counts[user_counts >= 50].index

# Estrai i dati per questi utenti
facebook_users_50 = facebook[facebook['from_id'].isin(users_at_least_50)]

# Calcola il numero di mesi attivi per ciascun utente
def calculate_active_months(user_df):
    min_date = user_df['created_time'].min()
    max_date = user_df['created_time'].max()
    active_months = pd.date_range(start=min_date, end=max_date, freq='MS').size
    return active_months

# Filtra gli utenti che hanno almeno 2 commenti al mese
def filter_users_by_comment_frequency(df):
    result = []
    for user_id in df['from_id'].unique():
        user_df = df[df['from_id'] == user_id]
        active_months = calculate_active_months(user_df)
        if active_months > 0 and (user_df.shape[0] / active_months) >= 2:
            result.append(user_id)
    return result

# Ottieni gli utenti che soddisfano entrambi i criteri
filtered_users = filter_users_by_comment_frequency(facebook_users_50)

# Visualizza il risultato
print(filtered_users)


['1.0208940605180856e+16', '1.0209739917950516e+16', '1.0102360325590636e+16', '1.0155536693250152e+16', '1.020942746975186e+16', '1.0214048042979962e+16', '1.0209460813783726e+16', '1.0155081790888128e+16', '2006554902703903.0', '1.021299493908631e+16', '1.0213641919822782e+16', '488594538147119.0', '105437636170410.0', '1.0154581420026806e+16', '1.0213401563134418e+16', '1.0212589536306972e+16', '1.020991609628418e+16', '401730193556681.0', '1.020895711214087e+16', '1.0212968549517498e+16', '1.0211630078322254e+16', '1.0212989251983996e+16', '1.0155020279864398e+16', '1.015558475558592e+16', '1.0211777378984418e+16', '1516434038396533.0', '1466533896765115.0', '1.0158899933160656e+16', '1.0155430388091358e+16', '743334822505396.0', '1.021256296812454e+16', '1459538480780500.0', '1607000802683496.0', '1.021383785652106e+16', '1.0207928444362636e+16', '777579679069218.0', '1658104577542051.0', '534632916742245.0', '1427044657383582.0', '1893213580935374.0', '1.0212150963242978e+16', '1

In [68]:
facebook

,Unnamed: 0,from_id,created_time,post_id,comment_id,toxicity_score,message_y,topic,user_id,from_name,message,created_at,likes_count,social
0,0,182901852246199.0,2017-05-31T17:26:44+0000,534632916742245_647700765435459,647700765435459_647714875434048,0.019714,"Wish it was true, unfortunately it isn't. The ...",Vaccines,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0155298044760734e+16,2017-05-31T17:27:59+0000,534632916742245_647700765435459,647700765435459_647715042100698,0.058929,Go Sweden!,Vaccines,NaN,NaN,NaN,NaN,NaN,NaN
2,2,673763449487076.0,2017-05-31T17:31:27+0000,534632916742245_647700765435459,647700765435459_647725668766302,0.049212,A Nobel thing to do for Medicine.Ã­Â Â¾Ã­Â´Â,Vaccines,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1.0155422882509632e+16,2017-05-31T17:56:16+0000,534632916742245_647700765435459,647700765435459_647753312096871,0.315403,Shazmeen Ahmed get that nurse to read this!,Vaccines,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1390446657702247.0,2017-05-31T17:59:01+0000,534632916742245_647700765435459,647700765435459_647754012096801,0.001588,Javier Olguin,Vaccines,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9200420,6654496,NaN,NaN,9299179711_10152794855754712,10152794855754712_10152794979334712,0.028782,NaN,News,1.020586e+16,Jeffrey Thomas,Bongino is a perpetual political candidate cam...,2014-10-27 13:48:10,0.0,Facebook
9200421,6654497,NaN,NaN,9299179711_10152794855754712,10152794855754712_10152795340649712,0.011037,NaN,News,1.015369e+16,Hetty Cassutto Haden,George Cassutto for your reading pleasure,2014-10-27 16:35:44,0.0,Facebook
9200422,6654498,NaN,NaN,9299179711_10152794855754712,10152794855754712_10152798874094712,0.016035,NaN,News,1.020974e+16,John Zinn,Mark,2014-10-29 03:07:56,0.0,Facebook
9200423,6654499,NaN,NaN,9299179711_10152794855754712,10152794855754712_10152796092169712,0.013675,NaN,News,1.015417e+16,George Cassutto,Delaney is awesome. I know him personally now.,2014-10-27 22:17:37,0.0,Facebook


Su facebook abbiamo 800k of users che hanno commentato più di 50 commenti 

In [67]:
len(facebook_users_50)/len(facebook['from_id'].unique())
len(facebook_users_50)

11451